# Well Placement and Control Optimization using a spatiotemporal proxy
### Misael M. Morales - 2024
***

In [40]:
import os
from time import time
from tqdm import tqdm
from multiprocessing import Pool

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec

from scipy.io import savemat, loadmat
from skimage.transform import resize
from sklearn.preprocessing import MinMaxScaler

import torch
import torch.nn as nn
from torch import einsum
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, TensorDataset, random_split

from neuralop.models import *
from transformers import Swinv2Config, Swinv2Model

from torchmetrics.image import StructuralSimilarityIndexMeasure as SSIMLoss
from torchmetrics.image import RelativeAverageSpectralError as RASELoss
from torchmetrics.image import SpatialCorrelationCoefficient as SCCLoss
from torchmetrics.image import SpectralAngleMapper as SAMLoss
from torchmetrics.image import SpectralDistortionIndex as SDILoss
from torchmetrics.image import TotalVariation as TVmetrics
from torchmetrics.image import UniversalImageQualityIndex as UIQILoss
from torchmetrics.image import VisualInformationFidelity as VIFLoss

from utils import check_torch
device = check_torch()

sec2year   = 365.25 * 24 * 60 * 60
psi2pascal = 6894.76
co2_rho    = 686.5266
mega       = 1e6

n_timesteps = 33
nx, ny, nz  = 100, 100, 11

indexMap = loadmat('data_100_100_11/G_cells_indexMap.mat', simplify_cells=True)['gci']
Grid = np.zeros((nx,ny,nz)).flatten(order='F')
Grid[indexMap] = 1
Grid = Grid.reshape(nx,ny,nz, order='F')
Tops = np.load('data_npy_100_100_11/tops_grid.npz')['tops']

------------------------------------------------------------
----------------------- VERSION INFO -----------------------
Torch version: 2.3.1+cu121 | Torch Built with CUDA? True
# Device(s) available: 1, Name(s): NVIDIA GeForce RTX 3080
------------------------------------------------------------


In [2]:
class FTMiONet(nn.Module):
    def __init__(self, hidden_1:int=16, hidden_2:int=32, hidden_3:int=64):
        super(FTMiONet, self).__init__()
        self.hidden_1 = hidden_1
        self.hidden_2 = hidden_2
        self.hidden_3 = hidden_3

        self.conv1 = nn.Conv3d(2, hidden_1, kernel_size=3, padding=1)
        self.conv2 = nn.Conv3d(hidden_1, hidden_2, kernel_size=3, padding=1)
        self.conv3 = nn.Conv3d(hidden_2, hidden_3, kernel_size=3, padding=1)
        self.norm1 = nn.InstanceNorm3d(hidden_1)
        self.norm2 = nn.InstanceNorm3d(hidden_2)
        self.norm3 = nn.InstanceNorm3d(hidden_3)
        self.pool = nn.MaxPool3d((1,1,2))
        self.gelu = nn.GELU()

        self.vitm = Swinv2Model(Swinv2Config(image_size=100, num_channels=hidden_3,
                                             embed_dim=96, num_heads=[2,4,8,16],
                                             output_attention=True,
                                             output_hidden_states=True))

        self.vitg = Swinv2Model(Swinv2Config(image_size=100, num_channels=hidden_3,
                                             embed_dim=96, num_heads=[2,4,8,16],
                                             output_attention=True,
                                             output_hidden_states=True))

        self.tel1 = nn.TransformerEncoderLayer(d_model=10, nhead=2, dim_feedforward=1024, activation='gelu', batch_first=True)
        self.tel2 = nn.TransformerEncoderLayer(d_model=160, nhead=8, dim_feedforward=1024, activation='gelu', batch_first=True)
        self.tel3 = nn.TransformerEncoderLayer(d_model=32, nhead=8, dim_feedforward=1024, activation='gelu', batch_first=True)

        self.trf1 = nn.TransformerEncoder(self.tel1, num_layers=4)
        self.trf2 = nn.TransformerEncoder(self.tel2, num_layers=4)
        self.trf3 = nn.TransformerEncoder(self.tel3, num_layers=4)

        self.fno = FNO(n_modes=(1,4), n_layers=2, norm='instance_norm',
                       in_channels=2, 
                       lifting_channels=hidden_1, 
                       hidden_channels=hidden_3, 
                       projection_channels=hidden_1,
                       out_channels=2)
        self.lift = nn.Linear(1920, 29128)

    def forward(self, x):
        xm, xg, xw, xc, xt = x

        zm = self.pool(self.norm1(self.gelu(self.conv1(xm))))
        zm = self.pool(self.norm2(self.gelu(self.conv2(zm))))
        zm = self.pool(self.norm3(self.gelu(self.conv3(zm)))).squeeze()
        mv = self.vitm(zm)
        zm = mv['reshaped_hidden_states'][-1].reshape(zm.shape[0], -1)

        zg = self.pool(self.norm1(self.gelu(self.conv1(xg))))
        zg = self.pool(self.norm2(self.gelu(self.conv2(zg))))
        zg = self.pool(self.norm3(self.gelu(self.conv3(zg)))).squeeze()
        gv = self.vitg(zg)
        zg = gv['reshaped_hidden_states'][-1].reshape(zg.shape[0], -1)

        zw = xw.view(xw.shape[0], -1)
        zw = self.trf1(zw)

        zc = xc.view(xc.shape[0], -1)
        zc = self.trf2(zc)

        zt = xt.view(xt.shape[0], -1)
        zt = self.trf3(zt)

        mg = torch.einsum('bp,bp->bp', zm, zg)
        wc = torch.einsum('bw,bc->bwc', zw, zc)
        zb = torch.einsum('bp,bwc->bwcp', mg, wc)
        zb = zb.reshape(zb.shape[0], 2, 5, 5, 32, 32, 384)
        zb = torch.einsum('blwwttp,blwwttp->blwtp', zb, zb)
        merge = torch.einsum('blwtp,bt->blwtp', zb, zt)
        merge = merge.permute(0,1,3,2,4).reshape(merge.shape[0], 2, 32, -1)
        zy = self.fno(merge)
        yy = self.lift(zy)
        
        return yy

In [7]:
class CustomDataset(Dataset):
    def __init__(self, data_folder:str='data_npy_100_100_11'):
        self.data_folder = data_folder
        
        self.x_folder = os.path.join(data_folder, 'inputs_rock_rates_locs_time')
        self.y_folder = os.path.join(data_folder, 'outputs_masked_pressure_saturation')

        self.x_file_list = os.listdir(self.x_folder)
        self.y_file_list = os.listdir(self.y_folder)

    def __len__(self):
        return len(self.x_file_list)
    
    def __getitem__(self, idx):
        x  = np.load(os.path.join(self.x_folder, self.x_file_list[idx]))
        y  = np.load(os.path.join(self.y_folder, self.y_file_list[idx]))

        xg = np.concatenate([np.expand_dims(Tops/(3000), 0), 
                             np.expand_dims(Grid, 0)], 
                             axis=0)

        xm = np.concatenate([np.expand_dims(x['poro']/(0.3),0), 
                             np.expand_dims(x['perm']/(3.3),0)], 
                             axis=0)
        
        xw = x['locs']           / (100)
        xc = x['ctrl'][1:]       * co2_rho*sec2year/mega/1e3/(25)
        xt = x['time'][1:]       / sec2year / (100)
        yp = y['pressure'][2:]   / psi2pascal / (1e4)
        ys = y['saturation'][2:] / 0.8
        yy = np.concatenate([np.expand_dims(yp,0), np.expand_dims(ys,0)], axis=0)

        xm = torch.tensor(xm, dtype=torch.float32, device=device)
        xg = torch.tensor(xg, dtype=torch.float32, device=device)
        xw = torch.tensor(xw, dtype=torch.float32, device=device)
        xc = torch.tensor(xc, dtype=torch.float32, device=device)
        xt = torch.tensor(xt, dtype=torch.float32, device=device)
        yy = torch.tensor(yy, dtype=torch.float32, device=device)

        return (xm, xg, xw, xc, xt), yy

In [ ]:
class CustomLoss(nn.Module):
    def __init__(self, alpha=0.85, beta=0.20):
        super(CustomLoss, self).__init__()
        self.mse = nn.MSELoss()
        self.mae = nn.L1Loss()
        self.ssim = SSIMLoss()
        self.alpha = alpha
        self.beta = beta

    def forward(self, true, pred):
        mse_loss = self.mse(true, pred)
        mae_loss = self.mae(true, pred)
        ssim_loss = 1 - self.ssim(true, pred)
        return self.alpha(self.beta*mse_loss + (1-self.beta)*mae_loss) + (1-self.alpah)*ssim_loss

In [39]:
dataset = CustomDataset()
trainset, testset  = random_split(dataset, [1172, 100])
trainset, validset = random_split(trainset, [1000, 172])

trainloader = DataLoader(trainset, batch_size=32, shuffle=True)
validloader = DataLoader(validset, batch_size=32, shuffle=False)

model = FTMiONet().to(device)
print('# Parameters: {:,}'.format(sum(p.numel() for p in model.parameters() if p.requires_grad)))
criterion = nn.MSELoss().to(device)
optimizer = optim.AdamW(model.parameters(), lr=1e-3, weight_decay=1e-8)

# Parameters: 114,008,580


In [ ]:
history = pd.read_csv('MiONet_losses.csv')

def plot_loss(history, figsize=(8,4)):
    plt.figure(figsize=figsize)
    plt.plot(history.index, history['train'], ls='-', label='Train')
    plt.plot(history.index, history['valid'], ls='-', label='Valid')
    plt.xlabel('Epoch'); plt.ylabel('Loss')
    plt.legend(); plt.grid(True, which='both')
    plt.tight_layout(); plt.savefig('loss.png'); plt.show()
    return None
plot_loss(history)

***
# END